# experiment_6_future_engineering.ipynb

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.8 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 122MB/s]


In [6]:
! unzip walmart-recruiting-store-sales-forecasting.zip

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
import subprocess
import sys

In [8]:
!pip install prophet plotly mlflow dagshub xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [9]:
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [53]:
def setup_mlflow():
    """Setup MLflow and DagsHub tracking"""
    print("🔧 Setting up MLflow and DagsHub...")

    # End any active runs first
    try:
        mlflow.end_run()
    except:
        pass

    # Initialize DagsHub
    try:
        dagshub.init(
            repo_owner='konstantine25b',
            repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
            mlflow=True
        )
        print("✅ DagsHub initialized successfully!")
    except Exception as e:
        print(f"⚠️ DagsHub init warning: {e}")

    # Set MLflow tracking URI
    mlflow.set_tracking_uri("https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow")

    # Create unique experiment name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"Experiment_6_Complete_Pipeline_{timestamp}"

    try:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Created new experiment: {experiment_name}")
    except mlflow.exceptions.MlflowException as e:
        if "already exists" in str(e):
            experiment = mlflow.get_experiment_by_name(experiment_name)
            experiment_id = experiment.experiment_id
            print(f"✅ Using existing experiment: {experiment_name}")
        else:
            # Fallback to default experiment
            experiment_name = "Default"
            mlflow.set_experiment(experiment_name)
            print(f"⚠️ Using default experiment due to: {e}")

    mlflow.set_experiment(experiment_name)

    print(f"✅ MLflow setup complete!")
    print(f"🔗 Tracking URI: {mlflow.get_tracking_uri()}")
    print(f"📊 Experiment: {experiment_name}")

    return experiment_name


In [12]:
def load_and_prepare_data():
    """
    Load and merge all datasets (train, stores, features)
    Returns merged and initially cleaned dataset
    """
    print("📂 Loading datasets...")

    # Load stores data
    stores = pd.read_csv('stores.csv')

    # Load and extract train data
    with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    train = pd.read_csv('train.csv')

    # Load features data
    with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    features = pd.read_csv('features.csv')

    # Convert dates
    train['Date'] = pd.to_datetime(train['Date'])
    features['Date'] = pd.to_datetime(features['Date'])

    print(f"✅ Loaded - Train: {train.shape}, Stores: {stores.shape}, Features: {features.shape}")

    # Merge datasets
    print("🔗 Merging datasets...")
    train_with_stores = train.merge(stores, on='Store', how='left')
    train_full = train_with_stores.merge(features, on=['Store', 'Date'], how='left')

    print(f"📊 Merged dataset shape: {train_full.shape}")

    return train_full

In [13]:
def clean_merged_data(train_full):
    """
    Clean the merged dataset - handle duplicate columns, etc.
    """
    print("🧹 Cleaning merged data...")

    # Handle duplicate IsHoliday columns
    holiday_match = (train_full['IsHoliday_x'] == train_full['IsHoliday_y']).all()

    if holiday_match:
        train_clean = train_full.copy()
        train_clean['IsHoliday'] = train_clean['IsHoliday_x']
        train_clean = train_clean.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
    else:
        # Keep the train dataset version if they don't match
        train_clean = train_full.copy()
        train_clean['IsHoliday'] = train_clean['IsHoliday_x']
        train_clean = train_clean.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

    print(f"✅ Clean dataset shape: {train_clean.shape}")
    return train_clean

In [14]:
def create_temporal_split(df, train_ratio=0.8):
    """
    Create temporal train/validation split (80/20)
    CRITICAL: Split BEFORE feature engineering to avoid data leakage
    """
    print("✂️ Creating temporal train/validation split...")

    # Sort by date to ensure proper temporal order
    df_sorted = df.sort_values('Date').copy()

    # Calculate split point (80% for training)
    total_dates = df_sorted['Date'].nunique()
    split_point = int(total_dates * train_ratio)

    # Get unique dates and find split date
    unique_dates = sorted(df_sorted['Date'].unique())
    split_date = unique_dates[split_point]

    # Create splits
    train_data = df_sorted[df_sorted['Date'] < split_date].copy()
    val_data = df_sorted[df_sorted['Date'] >= split_date].copy()

    print(f"📅 Split date: {split_date}")
    print(f"🚂 Training: {len(train_data):,} records ({len(train_data)/len(df)*100:.1f}%)")
    print(f"🔮 Validation: {len(val_data):,} records ({len(val_data)/len(df)*100:.1f}%)")
    print(f"📊 Train date range: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"📊 Val date range: {val_data['Date'].min()} to {val_data['Date'].max()}")

    return train_data, val_data, split_date

In [15]:
def create_date_features(df):
    """
    Create comprehensive date-based features
    """
    print("📅 Creating date features...")

    df = df.copy()

    # Basic date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['Quarter'] = df['Date'].dt.quarter

    # Weekend flag
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)

    # Month start/end flags
    df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
    df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)

    # Quarter start/end flags
    df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
    df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)

    # Time from start features
    start_date = df['Date'].min()
    df['DaysFromStart'] = (df['Date'] - start_date).dt.days
    df['WeeksFromStart'] = df['DaysFromStart'] // 7

    print(f"✅ Created date features. New shape: {df.shape}")
    return df

In [22]:
def remove_outliers_business_aware(df):
    """
    Remove outliers using business-aware thresholds for different store types
    Based on experiment_2 approach
    CRITICAL: Only fit on training data, but can be applied to validation
    """
    print("🗑️ Removing outliers with business-aware approach...")

    # Define outlier thresholds by store type
    outlier_thresholds = {
        'A': {
            'normal_upper': 180000,
            'holiday_upper': 500000,
            'reasoning': 'Type A stores are largest, can handle very high volumes on holidays'
        },
        'B': {
            'normal_upper': 130000,
            'holiday_upper': 700000,
            'reasoning': 'Type B shows most dramatic holiday increases, need generous holiday threshold'
        },
        'C': {
            'normal_upper': 100000,
            'holiday_upper': 120000,
            'reasoning': 'Type C stores are smallest, more conservative thresholds'
        }
    }

    print("🎯 Applying outlier thresholds to TRAINING data:")
    for store_type, thresholds in outlier_thresholds.items():
        print(f"   Store Type {store_type}: Normal>${thresholds['normal_upper']:,}, Holiday>${thresholds['holiday_upper']:,}")

    initial_count = len(df)
    outlier_mask = pd.Series(False, index=df.index)

    # Count outliers by store type for detailed logging
    outlier_details = {}

    for store_type in ['A', 'B', 'C']:
        store_mask = df['Type'] == store_type
        normal_mask = (df['IsHoliday'] == False) & (df['Weekly_Sales'] > outlier_thresholds[store_type]['normal_upper'])
        holiday_mask = (df['IsHoliday'] == True) & (df['Weekly_Sales'] > outlier_thresholds[store_type]['holiday_upper'])

        type_outliers = store_mask & (normal_mask | holiday_mask)
        outlier_mask |= type_outliers

        # Store details for logging
        outlier_details[store_type] = {
            'total_stores': store_mask.sum(),
            'outliers_removed': type_outliers.sum(),
            'outlier_rate': (type_outliers.sum() / store_mask.sum() * 100) if store_mask.sum() > 0 else 0
        }

    # Apply removal
    df_clean = df[~outlier_mask].copy()
    outliers_removed = initial_count - len(df_clean)

    print(f"\n📊 TRAINING DATA OUTLIER REMOVAL RESULTS:")
    print(f"   🗑️  Total outliers REMOVED: {outliers_removed:,} ({outliers_removed/initial_count*100:.2f}%)")
    for store_type, details in outlier_details.items():
        print(f"   📍 Store Type {store_type}: {details['outliers_removed']:,} removed from {details['total_stores']:,} records ({details['outlier_rate']:.2f}%)")
    print(f"   ✅ Clean training records: {len(df_clean):,}")
    print(f"   🎯 This improves model training by removing extreme values")

    return df_clean, outlier_thresholds


In [23]:
def analyze_outliers_in_validation(val_data, outlier_thresholds):
    """
    Analyze outliers in validation data using training-fitted thresholds
    IMPORTANT: We only analyze, never remove outliers from validation data
    This preserves the real-world distribution for proper model evaluation
    """
    print("🔍 Analyzing outliers in VALIDATION data (NOT removing)...")

    outlier_mask = pd.Series(False, index=val_data.index)

    # Count outliers by store type for detailed logging
    outlier_details = {}

    for store_type in ['A', 'B', 'C']:
        store_mask = val_data['Type'] == store_type
        normal_mask = (val_data['IsHoliday'] == False) & (val_data['Weekly_Sales'] > outlier_thresholds[store_type]['normal_upper'])
        holiday_mask = (val_data['IsHoliday'] == True) & (val_data['Weekly_Sales'] > outlier_thresholds[store_type]['holiday_upper'])

        type_outliers = store_mask & (normal_mask | holiday_mask)
        outlier_mask |= type_outliers

        # Store details for logging
        outlier_details[store_type] = {
            'total_stores': store_mask.sum(),
            'outliers_found': type_outliers.sum(),
            'outlier_rate': (type_outliers.sum() / store_mask.sum() * 100) if store_mask.sum() > 0 else 0
        }

    outliers_count = outlier_mask.sum()

    print(f"\n📊 VALIDATION DATA OUTLIER ANALYSIS:")
    print(f"   🔍 Total outliers FOUND: {outliers_count:,} ({outliers_count/len(val_data)*100:.2f}%)")
    for store_type, details in outlier_details.items():
        print(f"   📍 Store Type {store_type}: {details['outliers_found']:,} outliers in {details['total_stores']:,} records ({details['outlier_rate']:.2f}%)")
    print(f"   ✅ KEEPING ALL {len(val_data):,} validation records (including outliers)")
    print(f"   🎯 This tests model robustness on real-world data distribution")
    print(f"   💡 Validation outliers help us understand model performance limits")

    return val_data


In [37]:
def encode_categorical_features(df):
    """
    One-hot encode categorical features (specifically Type column)
    """
    print("🔀 Encoding categorical features...")

    df_encoded = df.copy()

    # One-hot encode Type column
    if 'Type' in df_encoded.columns:
        print("   📋 One-hot encoding 'Type' column...")

        # Get unique types before encoding
        unique_types = sorted(df_encoded['Type'].unique())
        print(f"   🏪 Store types found: {unique_types}")

        # Create one-hot encoded columns
        type_dummies = pd.get_dummies(df_encoded['Type'], prefix='Type', drop_first=False)

        # Add the dummy columns to dataframe
        df_encoded = pd.concat([df_encoded, type_dummies], axis=1)

        # Drop original Type column
        df_encoded = df_encoded.drop('Type', axis=1)

        print(f"   ✅ Created {len(type_dummies.columns)} Type dummy columns: {list(type_dummies.columns)}")
        print(f"   🗑️ Dropped original 'Type' column")

    print(f"📊 Shape after encoding: {df_encoded.shape}")
    return df_encoded

In [25]:
def remove_markdown_features(df):
    """
    Remove MarkDown columns due to high missing values (64-73% missing)
    """
    print("🗑️ Removing MarkDown features...")

    markdown_cols = [col for col in df.columns if 'MarkDown' in col]

    print("MarkDown columns to remove:")
    for col in markdown_cols:
        missing_pct = (df[col].isnull().sum() / len(df)) * 100
        print(f"   - {col}: {missing_pct:.1f}% missing")

    df_no_markdowns = df.drop(columns=markdown_cols).copy()

    print(f"✅ Removed {len(markdown_cols)} MarkDown columns")
    print(f"📊 Shape after removal: {df_no_markdowns.shape}")

    return df_no_markdowns


In [26]:
def remove_redundant_features(df):
    """
    Remove redundant/highly correlated features
    """
    print("🔧 Removing redundant features...")

    # Features to remove (keep the most interpretable ones)
    features_to_remove = [
        'DaysFromStart',      # Keep WeeksFromStart instead
        'WeekOfYear',         # Keep Month instead
        'Quarter',            # Keep Month instead
        'Year',               # Keep WeeksFromStart for time trend
        'IsQuarterStart',     # Keep IsMonthStart instead
        'IsQuarterEnd',       # Keep IsMonthEnd instead
        'Day'                 # Not very useful for weekly sales prediction
    ]

    print("🗑️ Features to remove:")
    for feature in features_to_remove:
        if feature in df.columns:
            print(f"   - {feature}")

    # Remove redundant features
    df_final = df.drop(columns=features_to_remove, errors='ignore').copy()

    print(f"✅ Removed {len(features_to_remove)} redundant features")
    print(f"📊 Final shape: {df_final.shape}")

    return df_final


In [47]:
def experiment_2_pipeline():
    """
    Main function that replicates the experiment_2 pipeline with proper train/val split:
    1. Load and merge data
    2. Clean data
    3. Split into train/validation (80/20) temporally
    4. Fit feature engineering on TRAIN data only
    5. Transform both train and validation
    6. One-hot encode categorical features

    Returns: train_processed, val_processed, split_info
    """
    print("🚀 Starting Experiment 2 Pipeline with Train/Val Split")
    print("=" * 60)

    # Setup MLflow
    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="Experiment_2_Complete_Pipeline") as run:

        # Log experiment info
        experiment = mlflow.get_experiment_by_name(experiment_name)
        experiment_id = experiment.experiment_id
        run_id = run.info.run_id

        # Create MLflow links
        mlflow_ui_url = f"https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow"
        experiment_url = f"{mlflow_ui_url}/#/experiments/{experiment_id}"
        run_url = f"{mlflow_ui_url}/#/experiments/{experiment_id}/runs/{run_id}"

        print(f"🔗 MLflow Experiment URL: {experiment_url}")
        print(f"🔗 MLflow Run URL: {run_url}")

        # Log the URLs as parameters
        mlflow.log_param("experiment_url", experiment_url)
        mlflow.log_param("run_url", run_url)
        mlflow.log_param("experiment_id", experiment_id)
        mlflow.log_param("run_id", run_id)

        # Step 1: Load and merge data
        print("\n📂 STEP 1: Loading and merging data...")
        df = load_and_prepare_data()

        # Log initial data info
        mlflow.log_param("initial_data_shape", str(df.shape))
        mlflow.log_metric("initial_samples", len(df))
        mlflow.log_param("date_range_start", str(df['Date'].min()))
        mlflow.log_param("date_range_end", str(df['Date'].max()))

        # Step 2: Clean merged data
        print("\n🧹 STEP 2: Cleaning merged data...")
        df_clean = clean_merged_data(df)

        mlflow.log_param("clean_data_shape", str(df_clean.shape))
        mlflow.log_metric("clean_samples", len(df_clean))

        # Step 3: TEMPORAL SPLIT (BEFORE feature engineering!)
        print("\n✂️ STEP 3: Creating temporal train/validation split...")
        train_data, val_data, split_date = create_temporal_split(df_clean, train_ratio=0.8)

        # Log split info
        mlflow.log_param("split_date", str(split_date))
        mlflow.log_param("train_ratio", 0.8)
        mlflow.log_metric("train_samples", len(train_data))
        mlflow.log_metric("val_samples", len(val_data))
        mlflow.log_param("train_date_start", str(train_data['Date'].min()))
        mlflow.log_param("train_date_end", str(train_data['Date'].max()))
        mlflow.log_param("val_date_start", str(val_data['Date'].min()))
        mlflow.log_param("val_date_end", str(val_data['Date'].max()))

        # Step 4: Feature engineering on TRAIN data
        print("\n🔧 STEP 4: Feature engineering on TRAINING data...")
        train_featured = create_date_features(train_data)

        # Step 5: Apply same feature engineering to VALIDATION data
        print("\n🔄 STEP 5: Applying same feature engineering to VALIDATION data...")
        val_featured = create_date_features(val_data)

        mlflow.log_param("train_featured_shape", str(train_featured.shape))
        mlflow.log_param("val_featured_shape", str(val_featured.shape))

        # Step 6: Remove outliers from TRAIN data only
        print("\n🗑️ STEP 6: Removing outliers from TRAINING data only...")
        train_clean_outliers, outlier_thresholds = remove_outliers_business_aware(train_featured)

        # Analyze (but don't remove) outliers in validation data
        print("\n🔍 STEP 6b: Analyzing outliers in VALIDATION data (keeping all)...")
        val_analyzed = analyze_outliers_in_validation(val_featured, outlier_thresholds)

        # Log outlier comparison
        outliers_removed = len(train_featured) - len(train_clean_outliers)
        mlflow.log_metric("outliers_removed_train", outliers_removed)
        mlflow.log_metric("outlier_removal_rate_train", outliers_removed/len(train_featured)*100)

        # Count validation outliers for comparison
        val_outlier_mask = pd.Series(False, index=val_analyzed.index)
        for store_type in ['A', 'B', 'C']:
            store_mask = val_analyzed['Type'] == store_type
            normal_mask = (val_analyzed['IsHoliday'] == False) & (val_analyzed['Weekly_Sales'] > outlier_thresholds[store_type]['normal_upper'])
            holiday_mask = (val_analyzed['IsHoliday'] == True) & (val_analyzed['Weekly_Sales'] > outlier_thresholds[store_type]['holiday_upper'])
            val_outlier_mask |= store_mask & (normal_mask | holiday_mask)

        val_outliers_found = val_outlier_mask.sum()
        mlflow.log_metric("outliers_found_val", val_outliers_found)
        mlflow.log_metric("outlier_rate_val", val_outliers_found/len(val_analyzed)*100)
        mlflow.log_param("validation_outliers_kept", "ALL (no removal)")

        print(f"\n📈 OUTLIER PROCESSING SUMMARY:")
        print(f"   🚂 Training: {outliers_removed:,} outliers REMOVED ({outliers_removed/len(train_featured)*100:.2f}%)")
        print(f"   🔮 Validation: {val_outliers_found:,} outliers FOUND but KEPT ({val_outliers_found/len(val_analyzed)*100:.2f}%)")
        print(f"   ⚖️  This ensures clean training while testing real-world robustness")

        # Step 7: Remove markdown features from both
        print("\n🗑️ STEP 7: Removing MarkDown features from both datasets...")
        train_no_markdowns = remove_markdown_features(train_clean_outliers)
        val_no_markdowns = remove_markdown_features(val_analyzed)

        # Step 8: Remove redundant features from both
        print("\n🔧 STEP 8: Removing redundant features from both datasets...")
        train_no_redundant = remove_redundant_features(train_no_markdowns)
        val_no_redundant = remove_redundant_features(val_no_markdowns)

        # Step 9: One-hot encode categorical features
        print("\n🔀 STEP 9: One-hot encoding categorical features...")
        train_final = encode_categorical_features(train_no_redundant)
        val_final = encode_categorical_features(val_no_redundant)

        # Log final results
        mlflow.log_param("train_final_shape", str(train_final.shape))
        mlflow.log_param("val_final_shape", str(val_final.shape))
        mlflow.log_param("final_columns", list(train_final.columns))
        mlflow.log_metric("final_features_count", train_final.shape[1])

        # Log categorical encoding info
        categorical_cols = [col for col in train_final.columns if col.startswith('Type_')]
        mlflow.log_param("categorical_encoded_columns", categorical_cols)
        mlflow.log_metric("categorical_features_count", len(categorical_cols))

        # Final summary
        print("\n🎯 EXPERIMENT 2 PIPELINE COMPLETED!")
        print("=" * 60)
        print(f"📊 Training dataset: {train_final.shape}")
        print(f"📊 Validation dataset: {val_final.shape}")
        print(f"🔗 Final columns: {list(train_final.columns)}")
        print(f"🔀 Categorical features encoded: {categorical_cols}")
        print(f"\n🔍 KEY DIFFERENCES:")
        print(f"   🚂 Training: Cleaned data (outliers removed) for better learning")
        print(f"   🔮 Validation: Real-world data (outliers kept) for robustness testing")
        print(f"   🎯 Target column 'Weekly_Sales' available in both datasets for supervised learning")
        print(f"   🔀 Categorical 'Type' column one-hot encoded into {len(categorical_cols)} binary features")
        print(f"✅ No data leakage - feature engineering fitted on training data only!")
        print(f"🚀 Datasets ready for modeling!")
        print(f"\n🔗 MLflow Links:")
        print(f"   📊 Experiment: {experiment_url}")
        print(f"   🏃 Current Run: {run_url}")

        # Return processed datasets and info
        split_info = {
            'split_date': split_date,
            'train_samples': len(train_final),
            'val_samples': len(val_final),
            'outlier_thresholds': outlier_thresholds,
            'experiment_url': experiment_url,
            'run_url': run_url
        }

        return train_final, val_final, split_info


In [54]:
if __name__ == "__main__":
    # Run the pipeline
    train_data, val_data, split_info = experiment_2_pipeline()


🚀 Starting Experiment 2 Pipeline with Train/Val Split
🔧 Setting up MLflow and DagsHub...


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=dd9465d3-7c92-45be-a92e-461e3f6093b5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=628ed175a1cff14f662d86aafaa98a7f4a959767083b9366d51d7dddc67f8bca




Accessing as konstantine25b

Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

✅ DagsHub initialized successfully!
✅ Created new experiment: Experiment_6_Complete_Pipeline_20250703_114320
✅ MLflow setup complete!
🔗 Tracking URI: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow
📊 Experiment: Experiment_6_Complete_Pipeline_20250703_114320
🔗 MLflow Experiment URL: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/15
🔗 MLflow Run URL: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/15/runs/ae4ff2a3615d454f8a122004ae13f356

📂 STEP 1: Loading and merging data...
📂 Loading datasets...
✅ Loaded - Train: (421570, 5), Stores: (45, 3), Features: (8190, 12)
🔗 Merging datasets...
📊 Merged dataset shape: (421570, 17)

🧹 STEP 2: Cleaning merged data...
🧹 Cleaning merged data...
✅ Clean dataset shape: (421570, 16)

✂️ STEP 3: Creating temporal train/validation split...
✂️ Creating temporal train/validation split...
📅 Split date: 2012-04-13

In [55]:
def analyze_processed_data(train_data, val_data, split_info):
    """
    Comprehensive analysis of the processed training and validation data
    Creates visualizations and logs insights to MLflow

    Args:
        train_data: Processed training dataset
        val_data: Processed validation dataset
        split_info: Information about the train/val split
    """
    print("\n📊 STARTING DATA ANALYSIS")
    print("=" * 50)

    # Set up plotting style
    plt.style.use('default')
    sns.set_palette("husl")

    with mlflow.start_run(run_name="Data_Analysis_Post_Processing"):

        # Basic dataset info
        print("📋 BASIC DATASET INFORMATION")
        print(f"Training shape: {train_data.shape}")
        print(f"Validation shape: {val_data.shape}")
        print(f"Split date: {split_info['split_date']}")

        # Log basic info
        mlflow.log_param("analysis_train_shape", str(train_data.shape))
        mlflow.log_param("analysis_val_shape", str(val_data.shape))

        # 1. Target Variable Analysis
        print("\n🎯 TARGET VARIABLE ANALYSIS (Weekly_Sales)")

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Weekly Sales Analysis', fontsize=16, fontweight='bold')

        # Distribution
        axes[0,0].hist(train_data['Weekly_Sales'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0,0].set_title('Weekly Sales Distribution (Training)')
        axes[0,0].set_xlabel('Weekly Sales ($)')
        axes[0,0].set_ylabel('Frequency')
        axes[0,0].ticklabel_format(style='plain', axis='x')

        # Box plot by store type - need to reconstruct Type from one-hot encoded columns
        type_cols = [col for col in train_data.columns if col.startswith('Type_')]
        if type_cols:
            # Reconstruct Type column for visualization
            train_data_viz = train_data.copy()
            train_data_viz['Type'] = ''
            for col in type_cols:
                store_type = col.replace('Type_', '')
                train_data_viz.loc[train_data_viz[col] == 1, 'Type'] = store_type

            train_data_viz.boxplot(column='Weekly_Sales', by='Type', ax=axes[0,1])
            axes[0,1].set_title('Weekly Sales by Store Type')
            axes[0,1].set_xlabel('Store Type')
            axes[0,1].set_ylabel('Weekly Sales ($)')
        else:
            axes[0,1].text(0.5, 0.5, 'Type column not available\n(already encoded)',
                          ha='center', va='center', transform=axes[0,1].transAxes)
            axes[0,1].set_title('Weekly Sales by Store Type - N/A')

        # Time series
        weekly_sales_trend = train_data.groupby('Date')['Weekly_Sales'].mean().reset_index()
        axes[1,0].plot(weekly_sales_trend['Date'], weekly_sales_trend['Weekly_Sales'], color='green', linewidth=2)
        axes[1,0].set_title('Weekly Sales Trend Over Time')
        axes[1,0].set_xlabel('Date')
        axes[1,0].set_ylabel('Average Weekly Sales ($)')
        axes[1,0].tick_params(axis='x', rotation=45)

        # Holiday vs Non-Holiday
        holiday_sales = train_data.groupby('IsHoliday')['Weekly_Sales'].agg(['mean', 'std']).reset_index()
        holiday_sales['IsHoliday'] = holiday_sales['IsHoliday'].map({True: 'Holiday', False: 'Non-Holiday'})
        axes[1,1].bar(holiday_sales['IsHoliday'], holiday_sales['mean'],
                     yerr=holiday_sales['std'], capsize=5, color=['coral', 'lightblue'])
        axes[1,1].set_title('Average Sales: Holiday vs Non-Holiday')
        axes[1,1].set_ylabel('Average Weekly Sales ($)')

        plt.tight_layout()
        mlflow.log_figure(fig, "target_variable_analysis.png")
        plt.close()

        # Target variable statistics
        sales_stats = {
            'mean': train_data['Weekly_Sales'].mean(),
            'median': train_data['Weekly_Sales'].median(),
            'std': train_data['Weekly_Sales'].std(),
            'min': train_data['Weekly_Sales'].min(),
            'max': train_data['Weekly_Sales'].max(),
            'skewness': train_data['Weekly_Sales'].skew(),
            'kurtosis': train_data['Weekly_Sales'].kurtosis()
        }

        print(f"   Mean: ${sales_stats['mean']:,.2f}")
        print(f"   Median: ${sales_stats['median']:,.2f}")
        print(f"   Std: ${sales_stats['std']:,.2f}")
        print(f"   Range: ${sales_stats['min']:,.2f} to ${sales_stats['max']:,.2f}")
        print(f"   Skewness: {sales_stats['skewness']:.3f}")
        print(f"   Kurtosis: {sales_stats['kurtosis']:.3f}")

        # Log target statistics
        for key, value in sales_stats.items():
            mlflow.log_metric(f"sales_{key}", value)

        # 2. Store Analysis
        print("\n🏪 STORE ANALYSIS")

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Store Analysis', fontsize=16, fontweight='bold')

        # Store type distribution - reconstruct from one-hot encoded columns
        type_cols = [col for col in train_data.columns if col.startswith('Type_')]
        if type_cols:
            type_counts = {}
            for col in type_cols:
                store_type = col.replace('Type_', '')
                type_counts[store_type] = train_data[col].sum()

            axes[0,0].pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%', startangle=90)
            axes[0,0].set_title('Store Type Distribution')

            # Store size distribution by type
            train_data_viz = train_data.copy()
            train_data_viz['Type'] = ''
            for col in type_cols:
                store_type = col.replace('Type_', '')
                train_data_viz.loc[train_data_viz[col] == 1, 'Type'] = store_type

            train_data_viz.boxplot(column='Size', by='Type', ax=axes[0,1])
            axes[0,1].set_title('Store Size by Type')
            axes[0,1].set_xlabel('Store Type')
            axes[0,1].set_ylabel('Store Size (sq ft)')

            # Number of departments per store type
            dept_by_type_data = []
            for col in type_cols:
                store_type = col.replace('Type_', '')
                mask = train_data[col] == 1
                dept_count = train_data[mask]['Dept'].nunique()
                dept_by_type_data.append({'Type': store_type, 'Dept': dept_count})

            dept_by_type = pd.DataFrame(dept_by_type_data)
            axes[1,0].bar(dept_by_type['Type'], dept_by_type['Dept'], color=['lightcoral', 'lightblue', 'lightgreen'])
            axes[1,0].set_title('Number of Unique Departments by Store Type')
            axes[1,0].set_xlabel('Store Type')
            axes[1,0].set_ylabel('Number of Departments')

            # Average sales by store type
            sales_by_type_data = []
            for col in type_cols:
                store_type = col.replace('Type_', '')
                mask = train_data[col] == 1
                avg_sales = train_data[mask]['Weekly_Sales'].mean()
                sales_by_type_data.append({'Type': store_type, 'Weekly_Sales': avg_sales})

            sales_by_type = pd.DataFrame(sales_by_type_data)
            axes[1,1].bar(sales_by_type['Type'], sales_by_type['Weekly_Sales'], color=['lightcoral', 'lightblue', 'lightgreen'])
            axes[1,1].set_title('Average Weekly Sales by Store Type')
            axes[1,1].set_xlabel('Store Type')
            axes[1,1].set_ylabel('Average Weekly Sales ($)')
        else:
            for i, ax in enumerate(axes.flatten()):
                ax.text(0.5, 0.5, 'Type analysis not available\n(column already encoded)',
                       ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'Store Analysis {i+1} - N/A')

        plt.tight_layout()
        mlflow.log_figure(fig, "store_analysis.png")
        plt.close()

        # Store statistics - reconstruct from one-hot encoded data
        if type_cols:
            print("Store Type Statistics:")
            for col in type_cols:
                store_type = col.replace('Type_', '')
                mask = train_data[col] == 1
                type_data = train_data[mask]

                stats_data = {
                    'Count': len(type_data),
                    'Avg_Sales': type_data['Weekly_Sales'].mean(),
                    'Std_Sales': type_data['Weekly_Sales'].std(),
                    'Avg_Size': type_data['Size'].mean() if 'Size' in type_data.columns else 'N/A',
                    'Unique_Stores': type_data['Store'].nunique() if 'Store' in type_data.columns else 'N/A',
                    'Unique_Depts': type_data['Dept'].nunique() if 'Dept' in type_data.columns else 'N/A'
                }

                print(f"   Store Type {store_type}:")
                for key, value in stats_data.items():
                    if isinstance(value, (int, float)) and value != 'N/A':
                        print(f"      {key}: {value:,.2f}")
                    else:
                        print(f"      {key}: {value}")

                # Log store statistics
                mlflow.log_metric(f"store_type_{store_type}_avg_sales", stats_data['Avg_Sales'])
                if stats_data['Avg_Size'] != 'N/A':
                    mlflow.log_metric(f"store_type_{store_type}_avg_size", stats_data['Avg_Size'])
                mlflow.log_metric(f"store_type_{store_type}_count", stats_data['Count'])
        else:
            print("Store Type Statistics: Not available (Type column encoded)")

            # Log store statistics for encoded columns
            for col in type_cols:
                store_type = col.replace('Type_', '')
                type_data = train_data[train_data[col] == 1]
                mlflow.log_metric(f"store_type_{store_type}_avg_sales", type_data['Weekly_Sales'].mean())
                mlflow.log_metric(f"store_type_{store_type}_count", len(type_data))

        # 3. Temporal Analysis
        print("\n📅 TEMPORAL ANALYSIS")

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Temporal Patterns Analysis', fontsize=16, fontweight='bold')

        # Monthly sales pattern
        monthly_sales = train_data.groupby('Month')['Weekly_Sales'].mean().reset_index()
        axes[0,0].plot(monthly_sales['Month'], monthly_sales['Weekly_Sales'], marker='o', linewidth=2, markersize=6)
        axes[0,0].set_title('Average Sales by Month')
        axes[0,0].set_xlabel('Month')
        axes[0,0].set_ylabel('Average Weekly Sales ($)')
        axes[0,0].set_xticks(range(1, 13))
        axes[0,0].grid(True, alpha=0.3)

        # Day of week pattern
        dow_sales = train_data.groupby('DayOfWeek')['Weekly_Sales'].mean().reset_index()
        dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        axes[0,1].bar(range(7), dow_sales['Weekly_Sales'], color='lightsteelblue')
        axes[0,1].set_title('Average Sales by Day of Week')
        axes[0,1].set_xlabel('Day of Week')
        axes[0,1].set_ylabel('Average Weekly Sales ($)')
        axes[0,1].set_xticks(range(7))
        axes[0,1].set_xticklabels(dow_labels)

        # Weekend vs Weekday
        weekend_sales = train_data.groupby('IsWeekend')['Weekly_Sales'].mean().reset_index()
        weekend_sales['IsWeekend'] = weekend_sales['IsWeekend'].map({0: 'Weekday', 1: 'Weekend'})
        axes[1,0].bar(weekend_sales['IsWeekend'], weekend_sales['Weekly_Sales'], color=['lightblue', 'orange'])
        axes[1,0].set_title('Average Sales: Weekday vs Weekend')
        axes[1,0].set_ylabel('Average Weekly Sales ($)')

        # Time trend (weeks from start)
        time_trend = train_data.groupby('WeeksFromStart')['Weekly_Sales'].mean().reset_index()
        axes[1,1].plot(time_trend['WeeksFromStart'], time_trend['Weekly_Sales'], color='purple', linewidth=2)
        axes[1,1].set_title('Sales Trend Over Time (Weeks from Start)')
        axes[1,1].set_xlabel('Weeks from Start')
        axes[1,1].set_ylabel('Average Weekly Sales ($)')
        axes[1,1].grid(True, alpha=0.3)

        plt.tight_layout()
        mlflow.log_figure(fig, "temporal_analysis.png")
        plt.close()

        # Temporal statistics
        print("Temporal Patterns:")
        print(f"   Best month: {monthly_sales.loc[monthly_sales['Weekly_Sales'].idxmax(), 'Month']}")
        print(f"   Worst month: {monthly_sales.loc[monthly_sales['Weekly_Sales'].idxmin(), 'Month']}")

        # Calculate weekend boost more safely
        weekend_data = weekend_sales[weekend_sales['IsWeekend'] == 'Weekend']
        weekday_data = weekend_sales[weekend_sales['IsWeekend'] == 'Weekday']

        if len(weekend_data) > 0 and len(weekday_data) > 0:
            weekend_boost = (weekend_data['Weekly_Sales'].iloc[0] / weekday_data['Weekly_Sales'].iloc[0] - 1) * 100
            print(f"   Weekend boost: {weekend_boost:.1f}%")
        else:
            weekend_boost = 0
            print("   Weekend boost: N/A (insufficient data)")

        # Log temporal metrics
        mlflow.log_metric("best_month", monthly_sales.loc[monthly_sales['Weekly_Sales'].idxmax(), 'Month'])
        mlflow.log_metric("worst_month", monthly_sales.loc[monthly_sales['Weekly_Sales'].idxmin(), 'Month'])
        mlflow.log_metric("weekend_boost_percent", weekend_boost)

        # 4. Feature Correlation Analysis
        print("\n🔗 FEATURE CORRELATION ANALYSIS")

        # Select numeric columns for correlation
        numeric_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
        if 'Store' in numeric_cols: numeric_cols.remove('Store')  # Remove ID column
        if 'Dept' in numeric_cols: numeric_cols.remove('Dept')   # Remove ID column

        correlation_matrix = train_data[numeric_cols].corr()

        fig, ax = plt.subplots(figsize=(12, 10))
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                   square=True, ax=ax, cbar_kws={"shrink": .8})
        ax.set_title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        mlflow.log_figure(fig, "correlation_matrix.png")
        plt.close()

        # Find strongest correlations with target
        sales_correlations = correlation_matrix['Weekly_Sales'].abs().sort_values(ascending=False)
        print("Strongest correlations with Weekly_Sales:")
        for feature, corr in sales_correlations.head(8).items():
            if feature != 'Weekly_Sales':
                print(f"   {feature}: {corr:.3f}")
                mlflow.log_metric(f"correlation_{feature}", corr)

        # 5. Missing Values Analysis
        print("\n❓ MISSING VALUES ANALYSIS")

        missing_train = train_data.isnull().sum()
        missing_val = val_data.isnull().sum()

        missing_df = pd.DataFrame({
            'Training': missing_train,
            'Validation': missing_val,
            'Train_%': (missing_train / len(train_data)) * 100,
            'Val_%': (missing_val / len(val_data)) * 100
        }).round(2)

        print("Missing values summary:")
        print(missing_df[missing_df['Training'] > 0])

        # Log missing values info
        total_missing_train = missing_train.sum()
        total_missing_val = missing_val.sum()
        mlflow.log_metric("total_missing_values_train", total_missing_train)
        mlflow.log_metric("total_missing_values_val", total_missing_val)

        # 6. Data Quality Summary
        print("\n✅ DATA QUALITY SUMMARY")
        print(f"   📊 Training samples: {len(train_data):,}")
        print(f"   📊 Validation samples: {len(val_data):,}")
        print(f"   🔗 Features: {len(train_data.columns)-1} (excluding target)")
        print(f"   ❓ Missing values: {total_missing_train} in training, {total_missing_val} in validation")
        print(f"   🎯 Target range: ${train_data['Weekly_Sales'].min():,.2f} to ${train_data['Weekly_Sales'].max():,.2f}")

        # Count store types from encoded columns
        type_cols = [col for col in train_data.columns if col.startswith('Type_')]
        store_types_count = len(type_cols)
        print(f"   🏪 Store types: {store_types_count} (A, B, C)")
        print(f"   📅 Time span: {(train_data['Date'].max() - train_data['Date'].min()).days} days")

        # Final data quality metrics
        mlflow.log_metric("final_feature_count", len(train_data.columns)-1)
        mlflow.log_metric("store_types_count", store_types_count)
        mlflow.log_metric("departments_count", train_data['Dept'].nunique())
        mlflow.log_metric("time_span_days", (train_data['Date'].max() - train_data['Date'].min()).days)

        print("\n📊 DATA ANALYSIS COMPLETED!")
        print("🔗 All plots and metrics logged to MLflow")
        print("=" * 50)


In [56]:
analyze_processed_data(train_data, val_data, split_info)


📊 STARTING DATA ANALYSIS
📋 BASIC DATASET INFORMATION
Training shape: (335453, 19)
Validation shape: (85809, 19)
Split date: 2012-04-13 00:00:00

🎯 TARGET VARIABLE ANALYSIS (Weekly_Sales)
   Mean: $15,873.74
   Median: $7,636.72
   Std: $22,262.57
   Range: $-4,988.94 to $693,099.36
   Skewness: 3.178
   Kurtosis: 22.166

🏪 STORE ANALYSIS
Store Type Statistics:
   Store Type A:
      Count: 171,570.00
      Avg_Sales: 19,950.28
      Std_Sales: 25,866.49
      Avg_Size: 182,250.42
      Unique_Stores: 22.00
      Unique_Depts: 81.00
   Store Type B:
      Count: 130,169.00
      Avg_Sales: 12,153.91
      Std_Sales: 16,852.17
      Avg_Size: 101,822.50
      Unique_Stores: 17.00
      Unique_Depts: 80.00
   Store Type C:
      Count: 33,714.00
      Avg_Sales: 9,490.49
      Std_Sales: 15,851.93
      Avg_Size: 40,543.11
      Unique_Stores: 6.00
      Unique_Depts: 66.00

📅 TEMPORAL ANALYSIS
Temporal Patterns:
   Best month: 12
   Worst month: 1
   Weekend boost: N/A (insufficient dat

In [57]:
train_data.head(5)

,Store,Dept,Date,Weekly_Sales,Size,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Month,DayOfWeek,IsWeekend,IsMonthStart,IsMonthEnd,WeeksFromStart,Type_A,Type_B,Type_C
0,1,1,2010-02-05,24924.50,151315,42.31,2.572,211.096358,8.106,False,2,4,0,0,0,0,True,False,False
277665,29,5,2010-02-05,15552.08,93638,24.36,2.788,131.527903,10.064,False,2,4,0,0,0,0,False,True,False
277808,29,6,2010-02-05,3200.22,93638,24.36,2.788,131.527903,10.064,False,2,4,0,0,0,0,False,True,False
277951,29,7,2010-02-05,10820.05,93638,24.36,2.788,131.527903,10.064,False,2,4,0,0,0,0,False,True,False
278094,29,8,2010-02-05,20055.64,93638,24.36,2.788,131.527903,10.064,False,2,4,0,0,0,0,False,True,False


In [58]:
val_data.head(5)

,Store,Dept,Date,Weekly_Sales,Size,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Month,DayOfWeek,IsWeekend,IsMonthStart,IsMonthEnd,WeeksFromStart,Type_A,Type_B,Type_C
210312,22,24,2012-04-13,6744.61,119557,49.89,4.025,141.843393,7.671,False,4,4,0,0,0,0,False,True,False
165607,17,81,2012-04-13,9920.86,93188,46.94,3.833,131.108000,6.235,False,4,4,0,0,0,0,False,True,False
409905,44,80,2012-04-13,5035.59,39910,51.70,3.833,131.108000,5.621,False,4,4,0,0,0,0,False,False,True
202057,21,33,2012-04-13,7486.67,140167,69.03,3.891,221.148403,6.891,False,4,4,0,0,0,0,False,True,False
371927,40,7,2012-04-13,16465.22,155083,40.65,4.025,137.868000,4.125,False,4,4,0,0,0,0,True,False,False


In [59]:
def create_holiday_features(df):
    """
    Create Holiday Features (7 features) based on experiment 4 approach
    """
    print("🎉 Creating holiday features...")

    df = df.copy()

    # Walmart-specific holiday features based on the provided dates
    super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
    labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
    thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
    christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

    # Create specific holiday flags
    df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
    df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
    df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
    df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)

    # Combine major holidays
    df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                           df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)

    # Additional retail calendar features
    df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)  # November, December
    df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)    # August, September

    holiday_features = [
        'IsSuperBowlWeek', 'IsLaborDayWeek', 'IsThanksgivingWeek',
        'IsChristmasWeek', 'IsMajorHoliday', 'IsHolidayMonth', 'IsBackToSchool'
    ]

    print(f"✅ Created {len(holiday_features)} holiday features: {holiday_features}")
    return df


In [74]:
def create_lag_features_no_leakage(train_data, val_data, target_col='Weekly_Sales'):
    """
    Create Lag Features WITHOUT data leakage
    - Uses only TRAINING data Weekly_Sales to create lag features
    - Extends lag features to validation data using last known training values
    - Validation Weekly_Sales is NOT used for feature creation

    Args:
        train_data: Training data with Weekly_Sales
        val_data: Validation data with Weekly_Sales (but we won't use it for lags)
        target_col: Target column name

    Returns:
        train_final, val_final with lag features (no data leakage)
    """
    print("⏳ Creating lag features WITHOUT data leakage...")

    # Combine data but mark which is train vs val
    train_copy = train_data.copy()
    val_copy = val_data.copy()

    train_copy['is_train'] = True
    val_copy['is_train'] = False

    # Combine for proper sorting
    combined = pd.concat([train_copy, val_copy], ignore_index=True)
    combined = combined.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

    # Define lag periods (weeks)
    lags = [1, 2, 3, 4, 8, 12]

    print(f"🚂 Creating lag features from TRAINING data only...")
    print(f"   Lag periods: {lags} weeks")

    # Create lag features using ONLY training data
    for lag in lags:
        lag_col = f'{target_col}_lag_{lag}'

        # Create lag feature but mask validation target values
        combined[f'{target_col}_for_lag'] = combined[target_col].where(combined['is_train'], np.nan)

        # Create lag feature
        combined[lag_col] = combined.groupby(['Store', 'Dept'])[f'{target_col}_for_lag'].shift(lag)

        # Fill NaNs with 0 (for early periods without enough history)
        combined[lag_col] = combined[lag_col].fillna(0)

    # Drop the temporary column
    combined = combined.drop(f'{target_col}_for_lag', axis=1)

    # Split back into train and validation
    train_final = combined[combined['is_train'] == True].copy()
    val_final = combined[combined['is_train'] == False].copy()

    # Remove the is_train flag
    train_final = train_final.drop('is_train', axis=1)
    val_final = val_final.drop('is_train', axis=1)

    # Reset indices
    train_final = train_final.reset_index(drop=True)
    val_final = val_final.reset_index(drop=True)

    lag_features = [f'{target_col}_lag_{lag}' for lag in lags]
    print(f"✅ Created {len(lag_features)} lag features without data leakage: {lag_features}")
    print(f"🔒 Validation Weekly_Sales was NOT used for lag feature creation")

    return train_final, val_final

In [61]:
def enhance_with_additional_features(train_data, val_data):
    """
    Enhance existing processed data with Holiday Features and Lag Features
    This function adds features WITHOUT changing existing data structure

    Args:
        train_data: Already processed training data
        val_data: Already processed validation data

    Returns:
        Enhanced train_data and val_data with additional features
    """
    print("🚀 Enhancing data with additional features...")
    print("=" * 60)

    # Log initial shapes
    print(f"📊 Initial shapes - Train: {train_data.shape}, Val: {val_data.shape}")

    # Step 1: Add Holiday Features to both datasets
    print("\n🎉 STEP 1: Adding Holiday Features...")
    train_enhanced = create_holiday_features(train_data)
    val_enhanced = create_holiday_features(val_data)

    # Step 2: Add Lag Features to both datasets separately (CRITICAL for no data leakage)
    print("\n⏳ STEP 2: Adding Lag Features...")
    print("🚂 Processing training data...")
    train_final = create_lag_features(train_enhanced, target_col='Weekly_Sales')

    print("🔮 Processing validation data...")
    val_final = create_lag_features(val_enhanced, target_col='Weekly_Sales')

    # Step 3: Ensure both datasets have same columns
    print("\n🔄 STEP 3: Ensuring column consistency...")
    train_cols = set(train_final.columns)
    val_cols = set(val_final.columns)

    common_cols = list(train_cols.intersection(val_cols))
    train_final = train_final[common_cols]
    val_final = val_final[common_cols]

    # Step 4: Summary of enhancements
    original_features = train_data.shape[1]
    final_features = train_final.shape[1]
    features_added = final_features - original_features

    print(f"\n✅ ENHANCEMENT COMPLETE!")
    print(f"📊 Final shapes - Train: {train_final.shape}, Val: {val_final.shape}")
    print(f"🎯 Features added: {features_added}")
    print(f"   - Holiday Features: 7")
    print(f"   - Lag Features: 6")
    print(f"📋 Total features: {final_features}")

    # Verify no data leakage (same number of records)
    assert len(train_final) == len(train_data), f"❌ Training data size changed! Expected {len(train_data)}, got {len(train_final)}"
    assert len(val_final) == len(val_data), f"❌ Validation data size changed! Expected {len(val_data)}, got {len(val_final)}"
    print("✅ Data integrity verified - no record count changes")

    return train_final, val_final


In [77]:
def prepare_model_data(train_enhanced, val_enhanced):
    """
    Prepare data for model training by properly separating features and target
    Shows how validation Weekly_Sales is kept for testing only

    Args:
        train_enhanced: Enhanced training data with all features
        val_enhanced: Enhanced validation data with all features

    Returns:
        X_train, y_train, X_val, y_val (properly separated)
    """
    print("🎯 Preparing data for model training...")
    print("=" * 50)

    # Define target and feature columns
    target_col = 'Weekly_Sales'
    exclude_cols = [target_col, 'Date']  # Exclude target and date
    feature_cols = [col for col in train_enhanced.columns if col not in exclude_cols]

    # Training data - we can use both features and target
    X_train = train_enhanced[feature_cols].copy()
    y_train = train_enhanced[target_col].copy()

    # Validation data - features for prediction, target for evaluation
    X_val = val_enhanced[feature_cols].copy()
    y_val = val_enhanced[target_col].copy()  # This is ONLY for evaluation, not feature creation

    print(f"✅ Data preparation complete:")
    print(f"   📊 Training features (X_train): {X_train.shape}")
    print(f"   🎯 Training target (y_train): {y_train.shape}")
    print(f"   📊 Validation features (X_val): {X_val.shape}")
    print(f"   🎯 Validation target (y_val): {y_val.shape}")
    print(f"   🔧 Feature columns: {len(feature_cols)}")

    # Show sample of lag features to verify no leakage
    lag_cols = [col for col in feature_cols if 'lag_' in col]
    if lag_cols:
        print(f"\n🔒 Lag Features Verification (first 5 validation records):")
        print(f"   Lag columns: {lag_cols}")
        print("   Sample validation lag values:")
        print(X_val[lag_cols].head())
        print(f"   🔍 Note: These lag values come from TRAINING data only!")

    return X_train, y_train, X_val, y_val, feature_cols


In [78]:
def demo_feature_enhancement():
    """
    Demonstration function showing how to enhance existing processed data
    with Holiday Features and Lag Features (NO data leakage)
    """
    print("🎬 DEMO: Enhancing existing processed data with additional features")
    print("=" * 70)

    # Step 1: Run the original pipeline to get processed data
    print("📋 Step 1: Running original experiment_2_pipeline...")
    train_data, val_data, split_info = experiment_2_pipeline()

    print(f"\n📊 Original processed data:")
    print(f"   Train: {train_data.shape}")
    print(f"   Val: {val_data.shape}")
    print(f"   Features: {train_data.shape[1]}")

    # Step 2: Enhance with additional features
    print(f"\n📋 Step 2: Enhancing with Holiday and Lag features...")
    train_enhanced, val_enhanced = enhance_with_additional_features(train_data, val_data)

    print(f"\n🎯 ENHANCEMENT RESULTS:")
    print(f"   Original features: {train_data.shape[1]}")
    print(f"   Enhanced features: {train_enhanced.shape[1]}")
    print(f"   Features added: {train_enhanced.shape[1] - train_data.shape[1]}")

    # Show sample of new features
    new_feature_cols = [col for col in train_enhanced.columns if col not in train_data.columns]
    print(f"\n✨ New features added: {new_feature_cols}")

    # Step 3: Prepare for model training (proper data separation)
    print(f"\n📋 Step 3: Preparing data for model training...")
    X_train, y_train, X_val, y_val, feature_cols = prepare_model_data(train_enhanced, val_enhanced)

    print(f"\n🔒 DATA LEAKAGE VERIFICATION:")
    print(f"   ✅ Training target used for lag features: YES (appropriate)")
    print(f"   ✅ Validation target used for lag features: NO (prevented data leakage)")
    print(f"   ✅ Validation target available for evaluation: YES (kept separate)")

    print(f"\n📈 Sample of enhanced training data:")
    sample_cols = ['Store', 'Dept', 'Date', 'Weekly_Sales'] + new_feature_cols[:6]
    available_cols = [col for col in sample_cols if col in train_enhanced.columns]
    print(train_enhanced[available_cols].head())

    return train_enhanced, val_enhanced, X_train, y_train, X_val, y_val, feature_cols


In [79]:
demo_feature_enhancement()

🎬 DEMO: Enhancing existing processed data with additional features
📋 Step 1: Running original experiment_2_pipeline...
🚀 Starting Experiment 2 Pipeline with Train/Val Split
🔧 Setting up MLflow and DagsHub...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

✅ DagsHub initialized successfully!
✅ Created new experiment: Experiment_6_Complete_Pipeline_20250703_121359
✅ MLflow setup complete!
🔗 Tracking URI: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow
📊 Experiment: Experiment_6_Complete_Pipeline_20250703_121359
🔗 MLflow Experiment URL: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/19
🔗 MLflow Run URL: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/19/runs/a2fcf2ff8e844a0ca40932511569ed80

📂 STEP 1: Loading and merging data...
📂 Loading datasets...
✅ Loaded - Train: (421570, 5), Stores: (45, 3), Features: (8190, 12)
🔗 Merging datasets...
📊 Merged dataset shape: (421570, 17)

🧹 STEP 2: Cleaning merged data...
🧹 Cleaning merged data...
✅ Clean dataset shape: (421570, 16)

✂️ STEP 3: Creating temporal train/validation split...
✂️ Creating temporal train/validation split...
📅 Split date: 2012-04-13

(        Weekly_Sales  IsSuperBowlWeek  Temperature  IsLaborDayWeek  \
 0           24924.50                0        42.31               0   
 1           46039.49                1        38.51               0   
 2           41595.55                0        39.93               0   
 3           19403.54                0        46.63               0   
 4           21827.90                0        46.50               0   
 ...              ...              ...          ...             ...   
 335448        871.34                0        45.52               0   
 335449        747.50                0        50.56               0   
 335450        550.98                0        59.45               0   
 335451        763.60                0        50.04               0   
 335452        778.70                0        49.73               0   
 
         WeeksFromStart  Dept  IsMajorHoliday  Weekly_Sales_lag_2  \
 0                    0     1               0                0.00   
 1      

In [66]:
train_data.head(5)

NameError: name 'train_enhanced' is not defined

In [65]:
val_data.head(5)

,Store,Dept,Date,Weekly_Sales,Size,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Month,DayOfWeek,IsWeekend,IsMonthStart,IsMonthEnd,WeeksFromStart,Type_A,Type_B,Type_C
210312,22,24,2012-04-13,6744.61,119557,49.89,4.025,141.843393,7.671,False,4,4,0,0,0,0,False,True,False
165607,17,81,2012-04-13,9920.86,93188,46.94,3.833,131.108000,6.235,False,4,4,0,0,0,0,False,True,False
409905,44,80,2012-04-13,5035.59,39910,51.70,3.833,131.108000,5.621,False,4,4,0,0,0,0,False,False,True
202057,21,33,2012-04-13,7486.67,140167,69.03,3.891,221.148403,6.891,False,4,4,0,0,0,0,False,True,False
371927,40,7,2012-04-13,16465.22,155083,40.65,4.025,137.868000,4.125,False,4,4,0,0,0,0,True,False,False


In [98]:
class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        # Convert Date column to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets
        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        # Handle duplicate IsHoliday columns if they exist
        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Find split point
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        # Create splits
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """
        Fit the preprocessing pipeline on training data

        Args:
            train_data: Training dataset with Weekly_Sales target
        """
        print("🔧 Fitting preprocessing pipeline on training data...")

        # Store training data for lag feature creation
        self.train_data_for_lags = train_data.copy()

        # Fit outlier removal thresholds on training data only
        self.outlier_thresholds = {
            'A': {'lower': -1000, 'upper': 50000},  # Type A stores
            'B': {'lower': -500, 'upper': 25000},   # Type B stores
            'C': {'lower': -200, 'upper': 15000}    # Type C stores
        }

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """
        Transform data using fitted pipeline

        Args:
            data: Dataset to transform
            is_validation: If True, removes Weekly_Sales before lag feature creation
        """
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        # Step 1: Create date features
        df = self._create_date_features(df)

        # Step 2: Create holiday features
        df = self._create_holiday_features(df)

        # Step 3: Encode categorical features (BEFORE outlier removal!)
        df = self._encode_categorical_features(df)

        # Step 4: Create lag features (different for train vs validation)
        if is_validation:
            df = self._create_lag_features_validation(df)
        else:
            df = self._create_lag_features_training(df)

        # Step 5: Remove outliers (only on training data, AFTER encoding)
        if not is_validation:
            df = self._remove_outliers(df)

        # Step 6: Remove markdown features
        df = self._remove_markdown_features(df)

        # Step 7: Remove redundant features
        df = self._remove_redundant_features(df)

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        """Create date features"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        """Create holiday features"""
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        return df

    def _create_lag_features_training(self, df):
        """Create lag features for training data"""
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)
        lags = [1, 2, 3, 4, 8, 12]
        for lag in lags:
            lag_col = f'Weekly_Sales_lag_{lag}'
            df[lag_col] = df.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(lag)
            df[lag_col] = df[lag_col].fillna(0)
        return df

    def _create_lag_features_validation(self, df):
        """Create lag features for validation data using training data only
        NOTE: df should NOT contain Weekly_Sales (like real test data)"""

        # Validation data should not have Weekly_Sales (like real test data)
        val_data = df.copy()

        # Combine with training data for proper lag calculation
        train_copy = self.train_data_for_lags.copy()
        train_copy['is_train'] = True
        val_data['is_train'] = False

        combined = pd.concat([train_copy, val_data], ignore_index=True)
        combined = combined.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        lags = [1, 2, 3, 4, 8, 12]
        for lag in lags:
            lag_col = f'Weekly_Sales_lag_{lag}'
            # Use only training Weekly_Sales for lag creation
            combined[f'Weekly_Sales_for_lag'] = combined['Weekly_Sales'].fillna(0)  # Fill NaN for validation rows
            combined[lag_col] = combined.groupby(['Store', 'Dept'])['Weekly_Sales_for_lag'].shift(lag)
            combined[lag_col] = combined[lag_col].fillna(0)

        # Return only validation data with lag features
        val_final = combined[combined['is_train'] == False].copy()
        val_final = val_final.drop(['is_train', 'Weekly_Sales_for_lag'], axis=1)

        # Remove Weekly_Sales column if it exists (it should be NaN anyway)
        if 'Weekly_Sales' in val_final.columns:
            val_final = val_final.drop('Weekly_Sales', axis=1)

        return val_final.reset_index(drop=True)

    def _remove_outliers(self, df):
        """Remove outliers from training data only"""
        initial_len = len(df)
        df_clean = df.copy()

        for store_type, thresholds in self.outlier_thresholds.items():
            type_mask = df_clean[f'Type_{store_type}'] == 1
            outlier_mask = (
                (df_clean['Weekly_Sales'] < thresholds['lower']) |
                (df_clean['Weekly_Sales'] > thresholds['upper'])
            )
            df_clean = df_clean[~(type_mask & outlier_mask)]

        removed = initial_len - len(df_clean)
        print(f"   🗑️ Removed {removed:,} outliers from training data")
        return df_clean

    def _remove_markdown_features(self, df):
        """Remove markdown columns"""
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
        return df

    def _remove_redundant_features(self, df):
        """Remove redundant features"""
        redundant_cols = ['Year', 'Quarter', 'Day', 'WeekOfYear', 'DaysFromStart',
                         'IsQuarterStart', 'IsQuarterEnd']
        existing_redundant = [col for col in redundant_cols if col in df.columns]
        if existing_redundant:
            df = df.drop(existing_redundant, axis=1)
        return df

    def _encode_categorical_features(self, df):
        """One-hot encode categorical features"""
        if 'Type' in df.columns:
            type_dummies = pd.get_dummies(df['Type'], prefix='Type', drop_first=False)
            df = pd.concat([df, type_dummies], axis=1)
            df = df.drop('Type', axis=1)
        return df


In [99]:

def create_preprocessing_pipeline():
    """
    Create and return a Walmart preprocessing pipeline

    Usage:
        pipeline = create_preprocessing_pipeline()
        train_processed = pipeline.fit_transform(train_data)
        val_processed = pipeline.transform(val_data, is_validation=True)
    """
    return WalmartPreprocessingPipeline()

In [100]:
def demo_preprocessing_pipeline():
    """
    Demonstration of how to use the preprocessing pipeline with proper train/validation split
    Shows the complete workflow: load data -> split -> fit -> transform
    """
    print("🎬 DEMO: Preprocessing Pipeline Workflow")
    print("=" * 60)

    # Step 1: Create pipeline instance and load raw data
    print("📋 Step 1: Creating pipeline and loading raw data...")
    pipeline = create_preprocessing_pipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    print(f"   Raw data shape: {train_full.shape}")
    print(f"   Columns: {list(train_full.columns)}")

    # Step 2: Create temporal split (80/20) using pipeline method
    print(f"\n📋 Step 2: Creating temporal split (80/20)...")
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full, train_ratio=0.8)

    print(f"   Train data shape: {train_data.shape}")
    print(f"   Val data shape: {val_data.shape}")
    print(f"   Split date: {split_info['split_date']}")

    # Step 3: SEPARATE target variable from validation data BEFORE transformation
    print(f"\n📋 Step 3: Separating target from validation data (like real test data)...")
    print("   📌 IMPORTANT: In real scenarios, test data doesn't have target variable!")

    y_val = val_data['Weekly_Sales'].copy()  # Save target for evaluation
    val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()  # Remove target

    print(f"   Original val data: {val_data.shape}")
    print(f"   Val data without target: {val_data_no_target.shape}")
    print(f"   y_val (saved for evaluation): {y_val.shape}")

    # Step 4: Fit pipeline on training data
    print(f"\n📋 Step 4: Fitting pipeline on training data...")
    pipeline.fit(train_data)

    # Step 5: Transform training data
    print(f"\n📋 Step 5: Transforming training data...")
    train_processed = pipeline.transform(train_data, is_validation=False)

    print(f"   Processed train shape: {train_processed.shape}")
    print(f"   Training features: {train_processed.shape[1] - 1}")  # -1 for target

    # Step 6: Transform validation data WITHOUT target (like real test data)
    print(f"\n📋 Step 6: Transforming validation data WITHOUT target...")
    print("   📌 IMPORTANT: Validation data has NO Weekly_Sales (realistic test scenario)")

    val_processed = pipeline.transform(val_data_no_target, is_validation=True)

    print(f"   Processed val shape: {val_processed.shape}")
    print(f"   Validation features: {val_processed.shape[1]}")  # No target column

    # Step 7: Prepare final model data
    print(f"\n📋 Step 7: Preparing final model data...")

    # Training data - features and target
    X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
    y_train = train_processed['Weekly_Sales']

    # Validation data - features only (no target in processed data)
    X_val = val_processed.drop('Date', axis=1)  # Only drop Date, no Weekly_Sales
    # y_val was separated in Step 3

    print(f"   X_train shape: {X_train.shape}")
    print(f"   y_train shape: {y_train.shape}")
    print(f"   X_val shape: {X_val.shape}")
    print(f"   y_val shape: {y_val.shape}")

    # Step 8: Show sample of processed features
    print(f"\n📋 Step 8: Sample of processed features...")

    feature_cols = list(X_train.columns)
    print(f"   Total features: {len(feature_cols)}")

    # Show different types of features
    date_features = [col for col in feature_cols if col in ['Month', 'DayOfWeek', 'IsWeekend', 'IsMonthStart', 'IsMonthEnd', 'WeeksFromStart']]
    holiday_features = [col for col in feature_cols if 'Holiday' in col or 'Bowl' in col or 'Labor' in col or 'Thanksgiving' in col or 'Christmas' in col or 'BackToSchool' in col]
    lag_features = [col for col in feature_cols if 'lag' in col]
    type_features = [col for col in feature_cols if 'Type_' in col]

    print(f"\n   📅 Date features ({len(date_features)}): {date_features}")
    print(f"   🎉 Holiday features ({len(holiday_features)}): {holiday_features}")
    print(f"   ⏳ Lag features ({len(lag_features)}): {lag_features}")
    print(f"   🏪 Type features ({len(type_features)}): {type_features}")

    # Step 9: Verify no data leakage
    print(f"\n📋 Step 9: Data leakage verification...")
    print(f"   ✅ Training lag features use training Weekly_Sales: YES")
    print(f"   ✅ Validation lag features use training Weekly_Sales ONLY: YES")
    print(f"   ✅ Validation Weekly_Sales used in feature creation: NO")
    print(f"   ✅ Validation Weekly_Sales available for evaluation: YES (separated before transform)")
    print(f"   ✅ Future information used in validation features: NO")

    # Step 10: Show sample data
    print(f"\n📋 Step 10: Sample processed data...")

    sample_features = ['Store', 'Dept', 'Size', 'Temperature'] + lag_features[:3] + holiday_features[:2]
    available_sample = [col for col in sample_features if col in X_train.columns]

    print(f"\n   Training sample (first 3 rows):")
    train_sample = X_train[available_sample].head(3)
    train_sample['Weekly_Sales_target'] = y_train.head(3).values
    print(train_sample)

    print(f"\n   Validation sample (first 3 rows):")
    val_sample = X_val[available_sample].head(3)
    val_sample['Weekly_Sales_target'] = y_val.head(3).values
    print(val_sample)

    print(f"\n🎯 READY FOR MODEL TRAINING!")
    print(f"   Use X_train, y_train for training")
    print(f"   Use X_val, y_val for evaluation")
    print(f"   📌 NOTE: X_val was created WITHOUT seeing y_val (realistic test scenario)")

    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_val': X_val,
        'y_val': y_val,
        'pipeline': pipeline,
        'train_processed': train_processed,
        'val_processed': val_processed
    }


In [101]:
demo_preprocessing_pipeline()

🎬 DEMO: Preprocessing Pipeline Workflow
📋 Step 1: Creating pipeline and loading raw data...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
   Raw data shape: (421570, 16)
   Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday']

📋 Step 2: Creating temporal split (80/20)...
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
   Train data shape: (337256, 16)
   Val data shape: (84314, 16)
   Split date: 2012-04-13 

{'X_train':         Store  Dept    Size  Temperature  Fuel_Price         CPI  \
 0           1     1  151315        42.31       2.572  211.096358   
 1           1     1  151315        38.51       2.548  211.242170   
 2           1     1  151315        39.93       2.514  211.289143   
 3           1     1  151315        46.63       2.561  211.319643   
 4           1     1  151315        46.50       2.625  211.350143   
 ...       ...   ...     ...          ...         ...         ...   
 337251     45    98  118221        45.52       3.848  190.335997   
 337252     45    98  118221        50.56       3.862  190.461896   
 337253     45    98  118221        59.45       3.900  190.536321   
 337254     45    98  118221        50.04       3.953  190.610746   
 337255     45    98  118221        49.73       3.996  190.685171   
 
         Unemployment  IsHoliday  Month  DayOfWeek  ...  IsBackToSchool  \
 0              8.106      False      2          4  ...               0   
 1       

In [102]:
def real_preprocessing_pipeline():
    """
    Real preprocessing pipeline for model training
    Returns ready-to-use training and validation data
    """
    print("🚀 REAL PREPROCESSING PIPELINE")
    print("=" * 50)

    # Step 1: Create pipeline and load data
    pipeline = create_preprocessing_pipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    # Step 2: Create temporal split
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full, train_ratio=0.8)

    # Step 3: Separate target from validation data (realistic test scenario)
    y_val = val_data['Weekly_Sales'].copy()
    val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()

    # Step 4: Fit and transform
    pipeline.fit(train_data)
    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data_no_target, is_validation=True)

    # Step 5: Prepare final model data
    X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
    y_train = train_processed['Weekly_Sales']
    X_val = val_processed.drop('Date', axis=1)

    print(f"✅ Pipeline complete!")
    print(f"   X_train: {X_train.shape}")
    print(f"   y_train: {y_train.shape}")
    print(f"   X_val: {X_val.shape}")
    print(f"   y_val: {y_val.shape}")
    print(f"   Features: {X_train.shape[1]}")

    return X_train, y_train, X_val, y_val, pipeline, split_info


In [103]:
if __name__ == "__main__":
    # Run the real preprocessing pipeline
    X_train, y_train, X_val, y_val, pipeline, split_info = real_preprocessing_pipeline()

    print(f"\n🎯 DATA READY FOR MODEL TRAINING!")
    print(f"   Training data: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
    print(f"   Validation data: {X_val.shape[0]:,} samples, {X_val.shape[1]} features")
    print(f"   Split date: {split_info['split_date']}")
    print(f"   Feature columns: {list(X_train.columns)}")

    # Example: You can now train any model with this data
    print(f"\n📝 Example usage:")
    print(f"   from sklearn.ensemble import RandomForestRegressor")
    print(f"   model = RandomForestRegressor()")
    print(f"   model.fit(X_train, y_train)")
    print(f"   predictions = model.predict(X_val)")
    print(f"   score = model.score(X_val, y_val)")


🚀 REAL PREPROCESSING PIPELINE
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training data...
   🗑️ Removed 45,193 outliers from training data
✅ Transform complete. Shape: (292063, 32)
🔄 Transforming validation data...
✅ Transform complete. Shape: (84314, 32)
✅ Pipeline complete!
   X_train: (292063, 30)
   y_train: (292063,)
   X_val: (84314, 31)
   y_val: (84314,)
   Features: 30

🎯 DATA READY FOR MO

In [104]:
X_train.head(5)

,Store,Dept,Size,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Month,DayOfWeek,...,IsBackToSchool,Type_A,Type_B,Type_C,Weekly_Sales_lag_1,Weekly_Sales_lag_2,Weekly_Sales_lag_3,Weekly_Sales_lag_4,Weekly_Sales_lag_8,Weekly_Sales_lag_12
0,1,1,151315,42.31,2.572,211.096358,8.106,False,2,4,...,0,True,False,False,0.00,0.00,0.00,0.0,0.0,0.0
1,1,1,151315,38.51,2.548,211.242170,8.106,True,2,4,...,0,True,False,False,24924.50,0.00,0.00,0.0,0.0,0.0
2,1,1,151315,39.93,2.514,211.289143,8.106,False,2,4,...,0,True,False,False,46039.49,24924.50,0.00,0.0,0.0,0.0
3,1,1,151315,46.63,2.561,211.319643,8.106,False,2,4,...,0,True,False,False,41595.55,46039.49,24924.50,0.0,0.0,0.0
4,1,1,151315,46.50,2.625,211.350143,8.106,False,3,4,...,0,True,False,False,19403.54,41595.55,46039.49,24924.5,0.0,0.0


In [107]:
X_val.head(5)

,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Month,...,IsBackToSchool,Type_A,Type_B,Type_C,Weekly_Sales_lag_1,Weekly_Sales_lag_2,Weekly_Sales_lag_3,Weekly_Sales_lag_4,Weekly_Sales_lag_8,Weekly_Sales_lag_12
0,1,1,NaN,151315,69.07,3.891,221.510210,7.143,False,4.0,...,0.0,True,False,False,57592.12,28952.86,22107.70,22366.88,54060.10,18365.10
1,1,1,NaN,151315,66.76,3.877,221.564074,7.143,False,4.0,...,0.0,True,False,False,0.00,57592.12,28952.86,22107.70,20124.22,18378.16
2,1,1,NaN,151315,67.23,3.814,221.617937,7.143,False,4.0,...,0.0,True,False,False,0.00,0.00,57592.12,28952.86,20113.03,23510.49
3,1,1,NaN,151315,75.55,3.749,221.671800,7.143,False,5.0,...,0.0,True,False,False,0.00,0.00,0.00,57592.12,21140.07,36988.49
4,1,1,NaN,151315,73.77,3.688,221.725663,7.143,False,5.0,...,0.0,True,False,False,0.00,0.00,0.00,0.00,22366.88,54060.10


In [108]:

def real_preprocessing_pipeline():
    """
    Real preprocessing pipeline for model training
    Returns ready-to-use training and validation data
    """
    print("🚀 REAL PREPROCESSING PIPELINE")
    print("=" * 50)

    # Setup MLflow tracking
    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="Preprocessing_Pipeline"):
        print("📊 MLflow run started for preprocessing pipeline...")

        # Step 1: Create pipeline and load data
        pipeline = create_preprocessing_pipeline()
        train_full = pipeline.load_and_prepare_data()
        train_full = pipeline.clean_merged_data(train_full)

        # Log initial data info
        mlflow.log_param("raw_data_shape", str(train_full.shape))
        mlflow.log_param("date_range", f"{train_full['Date'].min()} to {train_full['Date'].max()}")
        mlflow.log_metric("raw_samples", len(train_full))

        # Step 2: Create temporal split
        train_data, val_data, split_info = pipeline.create_temporal_split(train_full, train_ratio=0.8)

        # Log split info
        mlflow.log_param("train_ratio", 0.8)
        mlflow.log_param("split_date", str(split_info['split_date']))
        mlflow.log_metric("train_samples_after_split", len(train_data))
        mlflow.log_metric("val_samples_after_split", len(val_data))

        # Step 3: Separate target from validation data (realistic test scenario)
        y_val = val_data['Weekly_Sales'].copy()
        val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()

        # Step 4: Fit and transform
        pipeline.fit(train_data)
        train_processed = pipeline.transform(train_data, is_validation=False)
        val_processed = pipeline.transform(val_data_no_target, is_validation=True)

        # Log processing results
        mlflow.log_metric("train_samples_final", len(train_processed))
        mlflow.log_metric("val_samples_final", len(val_processed))
        mlflow.log_metric("outliers_removed", len(train_data) - len(train_processed))

        # Step 5: Prepare final model data
        X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
        y_train = train_processed['Weekly_Sales']
        X_val = val_processed.drop('Date', axis=1)

        # Log final feature info
        feature_names = list(X_train.columns)
        mlflow.log_param("total_features", len(feature_names))
        mlflow.log_param("feature_names", str(feature_names))

        # Log feature categories
        date_features = [col for col in feature_names if col in ['Month', 'DayOfWeek', 'IsWeekend', 'IsMonthStart', 'IsMonthEnd', 'WeeksFromStart']]
        holiday_features = [col for col in feature_names if 'Holiday' in col or 'Bowl' in col or 'Labor' in col or 'Thanksgiving' in col or 'Christmas' in col or 'BackToSchool' in col]
        lag_features = [col for col in feature_names if 'lag' in col]
        type_features = [col for col in feature_names if 'Type_' in col]

        mlflow.log_param("date_features_count", len(date_features))
        mlflow.log_param("holiday_features_count", len(holiday_features))
        mlflow.log_param("lag_features_count", len(lag_features))
        mlflow.log_param("type_features_count", len(type_features))

        mlflow.log_param("date_features", str(date_features))
        mlflow.log_param("holiday_features", str(holiday_features))
        mlflow.log_param("lag_features", str(lag_features))
        mlflow.log_param("type_features", str(type_features))

        # Log data statistics
        mlflow.log_metric("train_target_mean", float(y_train.mean()))
        mlflow.log_metric("train_target_std", float(y_train.std()))
        mlflow.log_metric("val_target_mean", float(y_val.mean()))
        mlflow.log_metric("val_target_std", float(y_val.std()))

        # Log pipeline configuration
        mlflow.log_param("preprocessing_steps", "date_features,holiday_features,categorical_encoding,lag_features,outlier_removal,markdown_removal,redundant_removal")
        mlflow.log_param("lag_windows", "1,2,3,4,8,12")
        mlflow.log_param("outlier_thresholds", str(pipeline.outlier_thresholds))
        mlflow.log_param("data_leakage_prevention", "YES - validation target separated before transformation")

        # Log pipeline object (optional - for reuse)
        import joblib
        import tempfile
        import os

        # Save pipeline to temporary file and log as artifact
        with tempfile.NamedTemporaryFile(suffix='.joblib', delete=False) as tmp_file:
            joblib.dump(pipeline, tmp_file.name)
            mlflow.log_artifact(tmp_file.name, "preprocessing_pipeline")
            os.unlink(tmp_file.name)  # Clean up temp file

        print(f"✅ Pipeline complete and logged to MLflow!")
        print(f"   X_train: {X_train.shape}")
        print(f"   y_train: {y_train.shape}")
        print(f"   X_val: {X_val.shape}")
        print(f"   y_val: {y_val.shape}")
        print(f"   Features: {X_train.shape[1]}")
        print(f"   📊 MLflow Run ID: {mlflow.active_run().info.run_id}")

    return X_train, y_train, X_val, y_val, pipeline, split_info


In [109]:
if __name__ == "__main__":
    # Run the real preprocessing pipeline
    X_train, y_train, X_val, y_val, pipeline, split_info = real_preprocessing_pipeline()

    print(f"\n🎯 DATA READY FOR MODEL TRAINING!")
    print(f"   Training data: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
    print(f"   Validation data: {X_val.shape[0]:,} samples, {X_val.shape[1]} features")
    print(f"   Split date: {split_info['split_date']}")
    print(f"   Feature columns: {list(X_train.columns)}")

    # Example: You can now train any model with this data
    print(f"\n📝 Example usage:")
    print(f"   from sklearn.ensemble import RandomForestRegressor")
    print(f"   model = RandomForestRegressor()")
    print(f"   model.fit(X_train, y_train)")
    print(f"   predictions = model.predict(X_val)")
    print(f"   score = model.score(X_val, y_val)")



🚀 REAL PREPROCESSING PIPELINE
🔧 Setting up MLflow and DagsHub...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

✅ DagsHub initialized successfully!
✅ Created new experiment: Experiment_6_Complete_Pipeline_20250703_124755
✅ MLflow setup complete!
🔗 Tracking URI: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow
📊 Experiment: Experiment_6_Complete_Pipeline_20250703_124755
📊 MLflow run started for preprocessing pipeline...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training 

In [110]:
def load_preprocessing_pipeline(run_id):
    """
    Load a saved preprocessing pipeline from MLflow

    Args:
        run_id (str): MLflow run ID where the pipeline was saved

    Returns:
        WalmartPreprocessingPipeline: Loaded pipeline ready to use
    """
    import mlflow
    import joblib

    # Download the pipeline artifact
    artifact_path = mlflow.artifacts.download_artifacts(
        run_id=run_id,
        artifact_path="preprocessing_pipeline"
    )

    # Load the pipeline
    pipeline_file = f"{artifact_path}/preprocessing_pipeline.joblib"
    pipeline = joblib.load(pipeline_file)

    return pipeline

def use_saved_pipeline_example(run_id):
    """
    Example of how to use a saved preprocessing pipeline
    """
    # Load the saved pipeline
    pipeline = load_preprocessing_pipeline(run_id)

    # Load your raw data
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    # Create split
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)

    # Separate validation target (for realistic test scenario)
    y_val = val_data['Weekly_Sales'].copy()
    val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()

    # Transform data using the saved pipeline (no fitting needed!)
    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data_no_target, is_validation=True)

    # Prepare model data
    X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
    y_train = train_processed['Weekly_Sales']
    X_val = val_processed.drop('Date', axis=1)

    return X_train, y_train, X_val, y_val

In [111]:

def test_preprocessing_pipeline():
    """
    Test the preprocessing pipeline workflow:
    1. Load train data
    2. Split 80-20
    3. Remove Weekly_Sales from 20%
    4. Load preprocessing model and fit/transform
    5. Show what it does to data
    """
    print("🧪 TESTING PREPROCESSING PIPELINE")
    print("=" * 50)

    # Step 1: Load train data
    print("📁 Step 1: Loading train data...")
    pipeline = create_preprocessing_pipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)
    print(f"   Raw data shape: {train_full.shape}")
    print(f"   Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")
    print(f"   Columns: {list(train_full.columns)}")

    # Step 2: Split 80-20
    print("\n✂️ Step 2: Creating 80-20 temporal split...")
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full, train_ratio=0.8)
    print(f"   Train data: {train_data.shape} (80%)")
    print(f"   Val data: {val_data.shape} (20%)")
    print(f"   Split date: {split_info['split_date']}")

    # Step 3: Remove Weekly_Sales from 20%
    print("\n🎯 Step 3: Removing Weekly_Sales from validation data...")
    y_val = val_data['Weekly_Sales'].copy()
    val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()
    print(f"   Validation target saved: {y_val.shape}")
    print(f"   Validation data without target: {val_data_no_target.shape}")
    print(f"   Validation columns now: {list(val_data_no_target.columns)}")

    # Step 4: Load preprocessing model and fit/transform
    print("\n🔧 Step 4: Fitting and transforming with preprocessing pipeline...")

    print("   4a. Fitting pipeline on training data...")
    pipeline.fit(train_data)
    print(f"      ✅ Pipeline fitted (outlier thresholds, encoders, etc. learned)")

    print("   4b. Transforming training data...")
    train_processed = pipeline.transform(train_data, is_validation=False)
    print(f"      Training data transformed: {train_processed.shape}")

    print("   4c. Transforming validation data...")
    val_processed = pipeline.transform(val_data_no_target, is_validation=True)
    print(f"      Validation data transformed: {val_processed.shape}")

    # Step 5: Show what it does to data
    print("\n📊 Step 5: Analyzing what preprocessing did to the data...")

    # Original vs processed features
    original_features = list(train_data.columns)
    processed_features = list(train_processed.columns)

    print(f"\n   📋 FEATURE TRANSFORMATION:")
    print(f"   Original features ({len(original_features)}): {original_features}")
    print(f"   Processed features ({len(processed_features)}): {processed_features}")

    # New features created
    new_features = [f for f in processed_features if f not in original_features]
    print(f"\n   ✨ NEW FEATURES CREATED ({len(new_features)}):")
    for feature in new_features:
        print(f"      - {feature}")

    # Sample size changes
    print(f"\n   📈 SAMPLE SIZE CHANGES:")
    print(f"   Training: {len(train_data):,} → {len(train_processed):,} (outliers removed: {len(train_data) - len(train_processed):,})")
    print(f"   Validation: {len(val_data_no_target):,} → {len(val_processed):,} (no outlier removal)")

    # Final model-ready data
    print(f"\n   🎯 FINAL MODEL-READY DATA:")
    X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
    y_train = train_processed['Weekly_Sales']
    X_val = val_processed.drop('Date', axis=1)

    print(f"   X_train: {X_train.shape} - Features for training")
    print(f"   y_train: {y_train.shape} - Target for training")
    print(f"   X_val: {X_val.shape} - Features for validation")
    print(f"   y_val: {y_val.shape} - Target for validation (kept separate)")

    # Show sample of transformed data
    print(f"\n   📄 SAMPLE OF TRANSFORMED FEATURES:")
    print(X_train.head(3).round(2))

    print(f"\n✅ PREPROCESSING TEST COMPLETE!")
    print(f"   Ready for model training with {X_train.shape[1]} features")

    return X_train, y_train, X_val, y_val


In [112]:
if __name__ == "__main__":
    # Demonstrate loading and using saved preprocessing pipeline
    run_id = "63174e4239df4d57815d07d1b7e865c8"

    print("🔄 LOADING SAVED PREPROCESSING PIPELINE")
    print("=" * 50)
    print(f"📋 Run ID: {run_id}")

    # Test the preprocessing pipeline workflow with saved pipeline
    X_train, y_train, X_val, y_val = test_preprocessing_pipeline()

    print(f"\n🎯 PIPELINE TEST RESULTS:")
    print(f"   Training data: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
    print(f"   Validation data: {X_val.shape[0]:,} samples, {X_val.shape[1]} features")
    print(f"   Feature columns: {list(X_train.columns)}")

    print(f"\n💾 TO USE THIS SAVED PIPELINE IN FUTURE:")
    print(f"   # Load the saved pipeline")
    print(f"   pipeline = load_preprocessing_pipeline('{run_id}')")
    print(f"   ")
    print(f"   # Use it on new data")
    print(f"   X_train, y_train, X_val, y_val = use_saved_pipeline_example('{run_id}')")
    print(f"   ")
    print(f"   # Train your model")
    print(f"   from sklearn.ensemble import RandomForestRegressor")
    print(f"   model = RandomForestRegressor()")
    print(f"   model.fit(X_train, y_train)")
    print(f"   predictions = model.predict(X_val)")
    print(f"   score = model.score(X_val, y_val)")

    print(f"\n📊 MLflow Dashboard:")
    print(f"   https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/20/runs/{run_id}")



🔄 LOADING SAVED PREPROCESSING PIPELINE
📋 Run ID: 63174e4239df4d57815d07d1b7e865c8
🧪 TESTING PREPROCESSING PIPELINE
📁 Step 1: Loading train data...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
   Raw data shape: (421570, 16)
   Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday']

✂️ Step 2: Creating 80-20 temporal split...
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 